In [50]:
!pip install langgraph langchain tavily-python openai




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.5 MB/s eta 0:00:00


In [52]:
from langgraph.graph import StateGraph, END
from pydantic import BaseModel
from tavily import TavilyClient
import os

# 👉 Set your Tavily API key
os.environ["TAVILY_API_KEY"] = "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"
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])







In [70]:
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pandas as pd  # Add pandas to handle datasets

# Define the CampaignState class
class CampaignState(BaseModel):
    goal: str
    strategy: Dict[str, str]
    search_results: Optional[str] = None
    email_text: Optional[str] = None
    chatbot_script: Optional[str] = None

    class Config:
        extra = "allow"  # 💡 allow extra fields if needed

# Step 1: Load your datasets (example for sales.csv and customer.csv)
sales_df = pd.read_csv('sales.csv')  # Adjust path as necessary
customer_df = pd.read_csv('customer.csv')  # Adjust path as necessary

# Step 2: Extract the necessary information from the datasets for the campaign
# For example, you might extract a goal from sales or some other relevant data

goal = "Launch a festive offer on sedans for families"  # Example goal, modify as per your logic

# Example strategy could be based on your data:
strategy = {"discount": "15%", "region": "West India"}

# Example search results, email text, and chatbot script (you can pull these from the datasets too if needed)
search_results = "Competitor campaigns highlight the festive season with family offers."
email_text = "Special Offer: 15% off on sedans for families in West India!"
chatbot_script = "🤖 Ask me about the 15% discount on sedans for families this festive season!"

# Step 3: Create the CampaignState instance using the extracted data
campaign_state = CampaignState(
    goal=goal,
    strategy=strategy,
    search_results=search_results,
    email_text=email_text,
    chatbot_script=chatbot_script
)

# Step 4: Use campaign_state as needed (e.g., for further processing or storing)
print(campaign_state)







goal='Launch a festive offer on sedans for families' strategy={'discount': '15%', 'region': 'West India'} search_results='Competitor campaigns highlight the festive season with family offers.' email_text='Special Offer: 15% off on sedans for families in West India!' chatbot_script='🤖 Ask me about the 15% discount on sedans for families this festive season!'


In [71]:
# 🕵️ Tavily search agent
def search_competitors(state: CampaignState) -> CampaignState:
    query = f"Marketing campaigns for: {state.goal}"
    response = tavily.search(query=query, search_depth="advanced")
    result_text = response["results"][0]["content"] if response["results"] else "No relevant campaigns found."
    state.search_results = f"🕵️ Competitor campaigns: {result_text}"
    return state

# 📧 Email generator
def generate_email(state: CampaignState) -> CampaignState:
    state.email_text = (
        f"📧 Special Offer: {state.goal} in {state.strategy['region']} "
        f"with {state.strategy['discount']} discount!"
    )
    return state

# 💬 Chatbot generator
def generate_chatbot(state: CampaignState) -> CampaignState:
    state.chatbot_script = (
        f"🤖 Hi! Interested in our new '{state.goal}'? Ask me anything!"
    )
    return state





In [72]:
def build_smart_campaign_workflow():
    workflow = StateGraph(CampaignState)

    workflow.add_node("search_agent", search_competitors)
    workflow.add_node("email_generator", generate_email)
    workflow.add_node("chatbot_generator", generate_chatbot)

    workflow.set_entry_point("search_agent")

    workflow.add_edge("search_agent", "email_generator")
    workflow.add_edge("email_generator", "chatbot_generator")
    workflow.add_edge("chatbot_generator", END)

    return workflow.compile()






In [79]:
workflow = build_smart_campaign_workflow()
initial_state = CampaignState(
    goal="Launch a targeted campaign for Sedans in Kolkata based on recent sales data",
    strategy={"region": "South India", "discount": "10%"}
)

final_state = workflow.invoke(initial_state)
from collections import OrderedDict
from pprint import pprint

# Convert final_state to a dict (if it's not already)
final_dict = dict(final_state)

# Reorder the keys so 'goal' and 'strategy' are first
ordered_output = OrderedDict([
    ('goal', final_dict.get('goal')),
    ('strategy', final_dict.get('strategy')),
    ('search_results', final_dict.get('search_results')),
    ('email_text', final_dict.get('email_text')),
    ('chatbot_script', final_dict.get('chatbot_script')),
])

# Print the result
pprint(ordered_output)







OrderedDict([('goal',
              'Launch a targeted campaign for Sedans in Kolkata based on '
              'recent sales data'),
             ('strategy', {'discount': '10%', 'region': 'South India'}),
             ('search_results',
              "🕵️ Competitor campaigns: Discover Kolkata's best-selling used "
              'sedans in 2024. The Honda City leads with a 22% share; Explore '
              'key market trends and buyer preferences.'),
             ('email_text',
              '📧 Special Offer: Launch a targeted campaign for Sedans in '
              'Kolkata based on recent sales data in South India with 10% '
              'discount!'),
             ('chatbot_script',
              "🤖 Hi! Interested in our new 'Launch a targeted campaign for "
              "Sedans in Kolkata based on recent sales data'? Ask me "
              'anything!')])


In [80]:
workflow = build_smart_campaign_workflow()
initial_state = CampaignState(
    goal="Promote a summer sale on SUVs for families in tier-1 cities",
    strategy={"region": "East India", "discount": "20%"}
)

final_state = workflow.invoke(initial_state)
from collections import OrderedDict
from pprint import pprint

# Convert final_state to a dict (if it's not already)
final_dict = dict(final_state)

# Reorder the keys so 'goal' and 'strategy' are first
ordered_output = OrderedDict([
    ('goal', final_dict.get('goal')),
    ('strategy', final_dict.get('strategy')),
    ('search_results', final_dict.get('search_results')),
    ('email_text', final_dict.get('email_text')),
    ('chatbot_script', final_dict.get('chatbot_script')),
])

# Print the result
pprint(ordered_output)

OrderedDict([('goal',
              'Promote a summer sale on SUVs for families in tier-1 cities'),
             ('strategy', {'discount': '20%', 'region': 'East India'}),
             ('search_results',
              '🕵️ Competitor campaigns: Offer package deals on family-friendly '
              'vehicles, emphasizing features like spacious interiors, safety, '
              'and entertainment systems. You could also'),
             ('email_text',
              '📧 Special Offer: Promote a summer sale on SUVs for families in '
              'tier-1 cities in East India with 20% discount!'),
             ('chatbot_script',
              "🤖 Hi! Interested in our new 'Promote a summer sale on SUVs for "
              "families in tier-1 cities'? Ask me anything!")])
